In [ ]:
# Ensure deps are available in the kernel
import sys, subprocess
subprocess.run([sys.executable, "-m", "pip", "install", "duckdb", "pandas"], check=False)

SyntaxError: invalid syntax (1519514483.py, line 2)

Instalei todas as libraries no venv, e importei com alias, tambem precisei restartar o kernel

Lembrar de executar os codigos com alias, apos importar com nome proprio

In [ ]:
import duckdb as ddb
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()
csv_path = os.getenv("CSV_PATH", "./data/cleaned_up_pandas.csv")

df = pd.read_csv(csv_path)
con = ddb.connect()

O path do arquivo csv pode ser problemático, checar e usar o ""../../" como wildcard para paths

In [ ]:
con.execute("""
  CREATE OR REPLACE VIEW v AS
  SELECT * FROM read_csv_auto("../../data/cleaned_up_pandas.csv", header=True)
""")

queries = {
  "bookings_per_hour": """
    SELECT strftime('%H', Time) AS hour,
           COUNT(DISTINCT REPLACE("Booking ID", '"', '')) AS unique_bookings
    FROM v
    WHERE Time IS NOT NULL AND Time <> ''
    GROUP BY 1
    ORDER BY hour;
  """,

  "hour_with_most_bookings": """
    SELECT strftime('%H', Time) AS hour,
           COUNT(DISTINCT REPLACE("Booking ID", '"', '')) AS unique_bookings
    FROM v
    WHERE Time IS NOT NULL AND Time <> ''
    GROUP BY 1
    ORDER BY unique_bookings DESC
    LIMIT 1;
  """,

  "bookings_per_weekday": """
    SELECT strftime('%w', Date) AS weekday_num,
           CASE strftime('%w', Date)
             WHEN '0' THEN 'Sunday' WHEN '1' THEN 'Monday' WHEN '2' THEN 'Tuesday'
             WHEN '3' THEN 'Wednesday' WHEN '4' THEN 'Thursday' WHEN '5' THEN 'Friday'
             WHEN '6' THEN 'Saturday' END AS weekday_name,
           COUNT(DISTINCT REPLACE("Booking ID", '"', '')) AS unique_bookings
    FROM v
    WHERE Date IS NOT NULL AND Date <> ''
    GROUP BY 1,2
    ORDER BY weekday_num;
  """,

  "month_with_most_bookings": """
    SELECT strftime('%Y-%m', Date) AS month,
           COUNT(DISTINCT REPLACE("Booking ID", '"', '')) AS bookings
    FROM v
    WHERE Date IS NOT NULL AND Date <> ''
    GROUP BY 1
    ORDER BY bookings DESC
    LIMIT 1;
  """,

  "booking_status_breakdown": """
    SELECT "Booking Status",
           COUNT(DISTINCT REPLACE("Booking ID", '"', '')) AS bookings
    FROM v
    GROUP BY 1
    ORDER BY bookings DESC;
  """,

  "top_customer_and_payment_methods": """
    WITH top_customer AS (
      SELECT REPLACE("Customer ID", '"', '') AS customer_id,
             COUNT(DISTINCT REPLACE("Booking ID", '"', '')) AS total_bookings
      FROM v
      GROUP BY 1
      ORDER BY total_bookings DESC
      LIMIT 1
    )
    SELECT t.customer_id,
           v."Payment Method",
           COUNT(DISTINCT REPLACE(v."Booking ID", '"', '')) AS bookings_for_method
    FROM v
    JOIN top_customer t
      ON REPLACE(v."Customer ID", '"', '') = t.customer_id
    GROUP BY t.customer_id, v."Payment Method"
    ORDER BY bookings_for_method DESC;
  """
}

Bad pipe message: %s [b'"Chromium";v="140", "Not=A?Brand";v="24", "Google Chrome"']
Bad pipe message: %s [b'="140"\r\nsec-ch-ua-mobile: ?0\r\nsec-ch-ua-platform: "Windows']
Bad pipe message: %s [b'\nUpgrade-Insecure-Requests: 1\r\nUs', b'-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.']
Bad pipe message: %s [b'0.0 Safari/537.36\r\nAccept: text/html,applicatio']
Bad pipe message: %s [b'xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q', b'.7\r\nSec-Fetch-Site: none\r\nSec-Fetch-Mode: navigate\r\nSec-Fetc']
Bad pipe message: %s [b'ol: max-age=0\r\nsec-ch-ua: "Chromium";v="140", "Not=A?Brand";v="24", "Google Chrome";v="140"\r\nsec-ch-ua-mobile: ?0\r\n']
Bad pipe message: %s [b'c-ch-ua-platform: "Windows"\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) A', b'leWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36\r\nAccept: tex